# Imports

In [1]:
import numpy as np


# Custom imports (i.e. own scripts)

In [2]:
"""

If you use this ipython notebook outside its original directory
please set the path to the COMPAS repository and uncomment the lines below
this way it automatically finds the custom scripts that we need to import

"""
#pathCOMPASrepo       = '/home/cneijssel/Documents/COMPASpop/'
#pathToPostProcessing = 'popsynth/Papers/NeijsselEtAl/PostProcessing/1_RatePerUnitSolarMass/
#import sys
#sys.path.append('athCOMPASrepo+pathToPostProcessing)

#The script we use to quickly write an h5file
import ClassCosmicIntegrator as CI


# Structure Of scripts

We will not go into detail into the different subroutines.
Please see 

<a href="./2_CosmicIntegration/Notes.html">
2 - Notes/derivations/tests/tricks
</a>
 
But to give a general idea  of the structure between the scripts.




Say you create an instance of the whole routine called Data

Data      = CI.CosmicIntegrator(...........)

Then you can acces info about the compas data by

- Data.COMPAS.someAttribute

- Data.MSSFR.someAttribue

More details on how to get the attributes and their explanation see at the end of this notebook.

For now let me give three examples of a useful way to create an instance of the ClassCosmicIntegrator

# Minimum instance of the ClassCosmicIntegrator

It might be that you don't want to look at any data but just want to test subroutines.
I recommend to but the verbose attribute to true to see what it tries to do.
It still does a lot of stuff from default settings but will tell you that it cannot function

In [3]:
Data      = CI.CosmicIntegrator(COMPASpath = None,  verbose=True)

Creating redshift shells for integral
Creating instance COMPAS class
Just to double check you create instance of ClassCOMPAS without path/Data
Remember to  self.setCOMPASDCOmask() and self.setCOMPASData()

Creating instance MSSFR class
  COMPAS data is not given given hence I set nothing
warning MSSFR instance without metallicityGrid
warning no normalisation of mass evolved assuming 1
cant do it because we have no Metallicity grid

cannot set 2D-array of rates
COMPAS data is empty (COMPAS.setCOMPASData) 


However this way you can call subroutines and test them

In [4]:
#we have no data so no metallicity grid since that is based on the data
print(Data.COMPAS.metallicityGrid)

None


In [5]:
#But you can access the functions
redshifts = np.array([0,1,2])
print(Data.MSSFR.SFR_Madau(redshifts))

[1.49614924e+07 8.66529022e+07 1.31859016e+08]


In [6]:
#removing instance so I can savely redo it in the next example
del Data

# Doing the full Cosmic Integration

To settings of the cosmic integrator can be divided into three topics
which essentially relate to the three classes. 
The settings can be more compressed but by following these steps it is the most robust
way to double check your input

## The universe

We need to define the redshift shells (see <a href="./2_CosmicIntegration/1_Intro.html">
1 - concept of cosmic integration
</a>), and the general properties of the universe.
The values noted here are the default values. So in principle if you like them
you do not even have to pass them


In [7]:
hubbleConstant  = 67.8 
omegaMatter     = 0.308
redshiftFirstSFR= 10. 
minRedshift     = 0.0
maxRedshift     = 2. 
nrRedshiftBins  = 20

#just to be sure I already give the path+filename of the Data here
pathCOMPASOutput = '/home/cneijssel/Documents/Projects/Data/CosmicInt/'
#NOTE THAT IT CURRENTLY ASSUMES THAT YOUR FILE IS CALLED COMPASOutput.h5!!!!

Data      = CI.CosmicIntegrator(COMPASpath      = pathCOMPASOutput,\
                                hubbleConstant  = hubbleConstant,\
                                omegaMatter     = omegaMatter,\
                                redshiftFirstSFR= redshiftFirstSFR,\
                                minRedshift     = minRedshift,\
                                maxRedshift     = maxRedshift,\
                                nrRedshiftBins  = nrRedshiftBins)

Remember to  self.setCOMPASDCOmask() and self.setCOMPASData()

cannot set 2D-array of rates
COMPAS data is empty (COMPAS.setCOMPASData) 


#### The warnings are because we have not set the info needed for the COMPAS data

## COMPAS data and systems of interest

The above are warnings because we have not defined everything we need yet.
We also need some data, from which we can calculate the metallicity grid.
Furthermore we need to know at which DCOs we are going to look. Hence we need a DCO mask
which corresponds to the doubleCompactObjects group in the COMPASOutput.h5.

There are two options to do this. 

- 1 You provide your own mask based on the doubleCOMPACT objects group.

    Data.COMPAS.DCOmask = maskDCO
    
    This might be useful if you want to look at a very specific group of DCOs.
    Note that this method is less tested.

- 2 As done in the Cosmic Integration paper. You pick a combination of often used
    choices and let it create its own mask.


In [8]:
#Commented are alternative options

types             = 'BBH'    #'BHNS'  'BNS'    
withinHubbleTime  = True     #False
optimistic        = False    #

#Note that when within Hubble Time False will not help much
#given that any system born with a redshift before the first stars gets a rate=0

Data.COMPAS.setCOMPASDCOmask(types            = types, \
                             withinHubbleTime = withinHubbleTime,\
                             optimistic       = optimistic)


Now we know which systems we are going to look at. The next step is to set the COMPAS data.
This means I will look at all the metallicities and store some other attributes such as the masses
and mass ratios etc. Most importantly I create a couple of 2D-arrays.
For example I calculate per system of the data, per redshift shell (defined before), the redshift of birth.
By precalculating this, we do not have to redo it when we want to switch MSSFR prescription but keep everything the same.
This saves a lot of time :)

In [9]:
Data.COMPAS.setCOMPASData()
Data.setBirthTimesAnd2Darrays()

## Choose your MSSFR

We now have set the Integrator class and the COMPAS data class.

We only need to know what prescription of MSSFR to use in the MSSFR class.


### SFR options

In [10]:
#For the star formation rate we have 4 preset functions
Data.MSSFR.SFRprescription = 'Madau et al. (2014)'
Data.MSSFR.SFRprescription = 'Madau et al. (2017)'
Data.MSSFR.SFRprescription = 'Strolger et al. (2004)'
Data.MSSFR.SFRprescription = 'Neijssel et al. (2019)'
#You can also define your custom function (in the same functional form as Madau et al')
Data.MSSFR.SFRprescription = 'Custom SFR'
#If you do this you need to give values for the constants in the function
Data.MSSFR.customSFR       = [1,2,3,4] #see script for more info

In [11]:
Data.MSSFR.SFRprescription = 'Neijssel et al. (2019)'

### Metallicity distribution options

The metallicity distribution divides into two types

- 1 A prescription based on the distribution of galaxy stellar masses (GSMF) and galaxy stellar mass to metallicity relations (ZM)

- 2 A redshift dependent log-normal distribution.

#### If you pick option 1

In [12]:
Data.MSSFR.Zprescription = 'ZM_GSMF'

#Pick a galaxy stellar mass distribution single/double relates to the shape of the schechter function
Data.MSSFR.GSMFprescription = 'Panter et al. (2004) Single'
                             #'Furlong et al. (2015) Single'
                             #'Furlong et al. (2015) Double'
Data.MSSFR.ZMprescription   = 'Ma et al. (2015)'
                             #'Langer et al. (2006)'
                             #'Langer et al. +offset (2006)'
                             # There are more but these are not functional yet given their shape
            

#### If you pick option 2

In [13]:
Data.MSSFR.Zprescription = 'logNormal'

# A preset lognormal distribution from Neijssel et al 2019
Data.MSSFR.logNormalPrescription = 'Neijssel Phenomenological'

#Or a custom lognormal prescription
Data.MSSFR.logNormalPrescription = 'Custom Phenomenological'
#If you do this give a list with the values for [Z0, alpha, sigma] (these are the neijssel et al values)
Data.MSSFR.customLogNormal       = [0.035,  -0.23 , 0.39]

# All good to go now Integrate!

In [14]:
Data.GWdetector_sensitivity='O1'  #'design
Data.GWdetector_snrThreshold=8

Data.cosmologicalIntegration()

# The main output

In [15]:
#All these classes store several types of information.
#But the main output is two 2D-arrays.

RateAtSource   = Data.PerSystemPerRedshift_ratesIntrinsic   #dN Gpc-3 per year 

RateAtObserver = Data.PerSystemPerRedshift_ratesObserved    #dN per year

The above 2D arrays are the rates per individual system per redshift
in a type of time frame.

In [16]:
print(RateAtSource.shape)


(20, 291929)


In [17]:
#each row is a redshift centered in the shell (first row lowest redshift)
print(Data.Shell_centerRedshift)

[0.05 0.15 0.25 0.35 0.45 0.55 0.65 0.75 0.85 0.95 1.05 1.15 1.25 1.35
 1.45 1.55 1.65 1.75 1.85 1.95]


In [18]:
#each column is the rate of a specific system (ordered the same as the masses/massratio seeds column etc)
print(len(Data.COMPAS.mChirp))

291929


In [19]:
#What is the rate of the highest chirpmass at the lowest redshift in the observerframe
maxChirpMass = np.max(Data.COMPAS.mChirp)
index        = np.where(Data.COMPAS.mChirp == maxChirpMass)[0]
rate         = RateAtObserver[0][index]
print(index)
print('Rate Chirpmass= %s [Msun] at redshift %s in observerframe=%s (dN/dyr) '\
      %(maxChirpMass, Data.Shell_centerRedshift[0], rate))


[247233]
Rate Chirpmass= 64.58209607249745 [Msun] at redshift 0.05 in observerframe=[0.] (dN/dyr) 


By slicing the data based on the Data.COMPAS attributes and looking at the correct rows
you can create the plots (see plotting routines for more info), or for plots on specific
subroutines see the Notes

In [20]:
#I am done here see ya!
Data.close()

AttributeError: 'CosmicIntegrator' object has no attribute 'close'